### Dependencies


In [1]:
import os
from glob import glob

PATH = os.path.join("evaluation_result", "sa")
result = [
    y for x in os.walk(PATH) for y in glob(os.path.join(x[0], "removed_pred_prob.npz"))
]

for rs in result:
    os.rename(rs, rs.replace("removed_pred_prob", "recovered_pred_prob"))

result = [
    y for x in os.walk(PATH) for y in glob(os.path.join(x[0], "removed_pred_score.npz"))
]

for rs in result:
    os.rename(rs, rs.replace("removed_pred_score", "recovered_pred_score"))

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from matplotlib import rc, rcParams
from cv_exp.eval import process_eval_rs, read_eval_rs
from cv_exp.utils import (
    plot_eval_rs,
    plot_sanity_check,
    plot_metric_for_gag,
    display_df_df,
)

from cases.imagenet_exp import settings as imagenet_settings
from cases.isic_exp import settings as isic_settings
from cases.places365_exp import settings as places365_settings

# from cases.coco_exp import settings as coco_settings

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

sns.set_theme(style="darkgrid")

rcParams["font.family"] = "Times New Roman"
rcParams["mathtext.default"] = "regular"
rc("font", weight="bold")

In [4]:
def c(a, b):
    return round(((a / b) - 1) * 100, 2)

In [5]:
import pandas as pd


def vs_func(metric_key, a, b_list, rs):
    original_list = rs[metric_key][2]
    kv = {}
    for o in original_list:
        kv[o[0]] = o[1]

    ratio = []
    for b in b_list:
        if kv.get(b) is None:
            b = f"{b}2"
        if kv.get(a) is None:
            a = f"{a}2"
        ratio.append(kv[b] / kv[a])

    return ratio


def vs_func_2(metric_key, rs):
    matrix = [
        vs_func(
            metric_key,
            "SmoothGrad",
            [
                "Pos SmoothGrad",
                "Neg SmoothGrad",
                "GAG-1",
                "Proposed_Guided SmoothGrad",
                "GAG-3",
            ],
            rs,
        ),
        vs_func(
            metric_key,
            "IG",
            [
                "Pos IG",
                "Neg IG",
                "Abs IG",
                "Proposed_Guided IG",
                "Abs Proposed_Guided IG",
            ],
            rs,
        ),
        vs_func(
            metric_key,
            "GuidedIG",
            [
                "Pos GuidedIG",
                "Neg GuidedIG",
                "Abs GuidedIG",
                "Proposed_Guided GuidedIG",
                "Abs Proposed_Guided GuidedIG",
            ],
            rs,
        ),
        vs_func(
            metric_key,
            "BlurIG",
            [
                "Pos BlurIG",
                "Neg BlurIG",
                "Abs BlurIG",
                "Proposed_Guided BlurIG",
                "Abs Proposed_Guided BlurIG",
            ],
            rs,
        ),
        vs_func(metric_key, "Both GB", [
                "GB", "Neg GB", "Abs GB", "GB", "GB"], rs),
        # vs_func(metric_key, 'Pos SmoothGrad', ['Neg SmoothGrad'], rs),
        # vs_func(metric_key, 'Pos IG', ['Neg IG'], rs),
        # vs_func(metric_key, 'Pos GuidedIG', ['Neg GuidedIG'], rs),
        # vs_func(metric_key, 'Pos BlurIG', ['Neg BlurIG'], rs),
        # vs_func(metric_key, 'Both GB', ['Neg GB'], rs),
    ]

    return np.array(matrix)


def get_df(rs):

    if rs.get("Log-Cosh Dice Loss") is not None:
        loss = vs_func_2("Log-Cosh Dice Loss", rs)
    if rs.get("MAE") is not None:
        mae = vs_func_2("MAE", rs)
    rcap = vs_func_2("Prob: M3", rs)
    dauc = vs_func_2("DAUC", rs)
    iauc = vs_func_2("IAUC", rs)

    dauc_avg = dauc.mean(0)
    iauc_avg = iauc.mean(0)
    if rs.get("MAE") is not None:
        mae_avg = mae.mean(0)
    if rs.get("Log-Cosh Dice Loss") is not None:
        loss_avg = loss.mean(0)
    rcap_avg = rcap.mean(0)

    data = []
    if rs.get("Log-Cosh Dice Loss") is not None:
        data.append(loss_avg)
    if rs.get("MAE") is not None:
        data.append(mae_avg)

    data.extend(
        [
            rcap_avg,
            dauc_avg,
            iauc_avg,
        ]
    )

    # data -= 1
    # data *= 100
    data = np.round(data, 2)
    data = np.rot90(np.fliplr(data))
    columns = []
    if rs.get("Log-Cosh Dice Loss") is not None:
        columns.append("Loss")

    if rs.get("MAE") is not None:
        columns.append("MAE")

    columns.extend(["RCAP", "DAUC", "IAUC"])

    index = ["+", "-", "A", "G", "GA"]

    df = pd.DataFrame(data, columns=columns, index=index)
    return df

### Run 1


### ImageNet


#### ImageNet - All


In [7]:
imagenet_result, imagenet_settings = read_eval_rs(
    imagenet_settings, "sa", "imagenet", "resnet50", "imagenet_seg"
)

_ = plot_eval_rs(
    *process_eval_rs(
        imagenet_result,
        imagenet_settings,
        k="sa",
        seg=True,
        force=False,
        exclude=["GAG-11", "GAG-12", "Seg"],
    ),
    sort_key="RCAP↑",
    ascending=False
)

  0%|          | 0/45 [00:00<?, ?it/s]

[(1, 'Guided AbsoluatGrad'), (2, 'GAG-4'), (3, 'GradCAM'), (4, 'GAG-5'), (5, 'SmoothGrad-SQ'), (6, 'GAG-6'), (7, 'VarGrad'), (8, 'Proposed_Guided SmoothGrad'), (9, 'GAG-7'), (10, 'GAG-8'), (11, 'GAG-9'), (12, 'GAG-1'), (13, 'GAG-10'), (14, 'Pos SmoothGrad'), (15, 'Guided Backpropagation'), (16, 'Abs Proposed_Guided GuidedIG'), (17, 'Proposed_Guided GuidedIG'), (18, 'Abs Proposed_Guided BlurIG'), (19, 'Proposed_Guided BlurIG'), (20, 'Neg SmoothGrad'), (21, 'Abs GuidedIG'), (22, 'Neg GB'), (23, 'Abs BlurIG'), (24, 'SmoothGrad'), (25, 'Abs GB'), (26, 'Guided IG'), (27, 'Blur IG'), (28, 'Abs Proposed_Guided IG'), (29, 'Abs IG'), (30, 'Pos GuidedIG'), (31, 'Proposed_Guided IG'), (32, 'Neg GuidedIG'), (33, 'Integrated Gradients'), (34, 'Pos BlurIG'), (35, 'Neg BlurIG'), (36, 'Neg IG'), (37, 'Vanilla Gradient'), (38, 'Both GB'), (39, 'GB-MP'), (40, 'SmoothGrad-MP'), (41, 'GuidedIG-MP'), (42, 'Pos IG'), (43, 'BlurIG-MP'), (44, 'IG-MP'), (45, 'SmoothGrad narrow')]


MAE↓,Log-Cosh Dice Loss↓,RCAP↑,DAUC↓,IAUC↑,Name
0.474767,0.267848,0.437721,0.104200,0.681461,Guided AbsoluatGrad
0.477539,0.247934,0.422528,0.105346,0.682472,GAG-4
0.533307,0.134827,0.405913,0.149310,0.690723,GradCAM
0.482750,0.240162,0.404181,0.106110,0.682881,GAG-5
0.487492,0.353824,0.383364,0.103200,0.681974,SmoothGrad-SQ
0.488934,0.238083,0.381494,0.106929,0.682545,GAG-6
0.487181,0.343419,0.380860,0.103676,0.681283,VarGrad
0.483810,0.351561,0.367186,0.094509,0.668490,Proposed_Guided SmoothGrad
0.494825,0.237912,0.357661,0.106905,0.683572,GAG-7
0.500605,0.238770,0.337917,0.107125,0.683852,GAG-8


#### EXP: Comparison with SOTA and Reversed


In [8]:
rs_imagenet1 = plot_eval_rs(
    *process_eval_rs(
        imagenet_result,
        imagenet_settings,
        k="sa",
        seg=True,
        force=False,
        include=[
            # "GradCAM",
            "GAG-3",
            "VarGrad",
            "SmoothGrad",
            "IG",
            "GB",
            "GuidedIG",
            "BlurIG",
            "Vanilla",
            # 'SmoothGrad-MP', ``
            # 'SmoothGrad narrow',
            # 'SmoothGrad-SQ',
            # 'IG-MP',
            # 'GB-MP',
            # 'GuidedIG-MP',
            # 'BlurIG-MP',
        ],
        # use_include_order=True,
    ),
    # sort_key=None,
    sort_key="RCAP↑",
    ascending=False,
    table_name="Comparison with SOTA and Reversed"
)

  0%|          | 0/8 [00:00<?, ?it/s]

[(1, 'Guided AbsoluatGrad'), (2, 'VarGrad'), (3, 'Guided Backpropagation'), (4, 'SmoothGrad'), (5, 'Guided IG'), (6, 'Blur IG'), (7, 'Integrated Gradients'), (8, 'Vanilla Gradient')]


MAE↓,Log-Cosh Dice Loss↓,RCAP↑,DAUC↓,IAUC↑,Name
0.474767,0.267848,0.437721,0.104200,0.681461,Guided AbsoluatGrad
0.487181,0.343419,0.380860,0.103676,0.681283,VarGrad
0.489085,0.382622,0.172062,0.074304,0.650321,Guided Backpropagation
0.499001,0.331017,0.106728,0.077568,0.604721,SmoothGrad
0.490389,0.355447,0.089705,0.069034,0.598204,Guided IG
0.495475,0.373354,0.085276,0.081145,0.556277,Blur IG
0.506464,0.337765,0.044868,0.085472,0.512035,Integrated Gradients
0.503979,0.345887,0.034005,0.129132,0.451049,Vanilla Gradient


#### EXP: Ablation Study for Different Gradient Process


In [9]:
rs_imagenet2 = plot_eval_rs(
    *process_eval_rs(
        imagenet_result,
        imagenet_settings,
        k="sa",
        seg=True,
        force=False,
        include=[
            "SmoothGrad",
            "Pos SmoothGrad",
            "Neg SmoothGrad",
            "GAG-1",
            "Proposed_Guided SmoothGrad",
            "GAG-3",
            "IG",
            "Pos IG",
            "Neg IG",
            "Abs IG",
            "Proposed_Guided IG",
            "Abs Proposed_Guided IG",
            "GuidedIG",
            "Pos GuidedIG",
            "Neg GuidedIG",
            "Abs GuidedIG",
            "Proposed_Guided GuidedIG",
            "Abs Proposed_Guided GuidedIG",
            "BlurIG",
            "Pos BlurIG",
            "Neg BlurIG",
            "Abs BlurIG",
            "Proposed_Guided BlurIG",
            "Abs Proposed_Guided BlurIG",
            "Both GB",
            "GB",
            "Neg GB",
            "Abs GB",
        ],
        use_include_order=True,
    ),
    sort_key=None,
    ascending=False,
    table_name="Ablation Study for Different Gradient Process"
)

  0%|          | 0/28 [00:00<?, ?it/s]

[(1, 'SmoothGrad'), (2, 'Pos SmoothGrad'), (3, 'Neg SmoothGrad'), (4, 'GAG-1'), (5, 'Proposed_Guided SmoothGrad'), (6, 'Pos IG'), (7, 'Neg IG'), (8, 'Abs IG'), (9, 'Proposed_Guided IG'), (10, 'Abs Proposed_Guided IG'), (11, 'Pos GuidedIG'), (12, 'Neg GuidedIG'), (13, 'Abs GuidedIG'), (14, 'Proposed_Guided GuidedIG'), (15, 'Abs Proposed_Guided GuidedIG'), (16, 'Pos BlurIG'), (17, 'Neg BlurIG'), (18, 'Abs BlurIG'), (19, 'Proposed_Guided BlurIG'), (20, 'Abs Proposed_Guided BlurIG'), (21, 'Both GB'), (22, 'Neg GB'), (23, 'Abs GB'), (24, 'Guided AbsoluatGrad'), (25, 'Guided Backpropagation'), (26, 'Integrated Gradients'), (27, 'Blur IG'), (28, 'Guided IG')]


MAE↓,Log-Cosh Dice Loss↓,RCAP↑,DAUC↓,IAUC↑,Name
0.499001,0.331017,0.106728,0.077568,0.604721,SmoothGrad
0.506236,0.292316,0.216659,0.091811,0.625991,Pos SmoothGrad
0.505258,0.293354,0.124949,0.090415,0.635824,Neg SmoothGrad
0.509218,0.246309,0.308988,0.107451,0.684256,GAG-1
0.483810,0.351561,0.367186,0.094509,0.668490,Proposed_Guided SmoothGrad
0.506077,0.345759,0.010087,0.085811,0.450250,Pos IG
0.506881,0.345093,0.035057,0.092424,0.444543,Neg IG
0.516156,0.303142,0.070783,0.082134,0.568458,Abs IG
0.491241,0.366674,0.061226,0.086608,0.510374,Proposed_Guided IG
0.491015,0.342585,0.077364,0.084995,0.537313,Abs Proposed_Guided IG


In [12]:
get_df(rs_imagenet2)

,Loss,MAE,RCAP,DAUC,IAUC
+,1.01,1.00,1.71,0.94,1.05
-,1.01,1.00,1.34,1.08,0.98
A,0.91,1.01,2.00,1.05,1.11
G,1.06,0.98,2.62,0.96,1.13
GA,0.97,0.97,2.86,0.99,1.15


In [21]:
display_df_df(get_df(rs_imagenet2))

,Loss,MAE,RCAP,DAUC,IAUC
+,1.010000,1.000000,1.710000,0.940000,1.050000
-,1.010000,1.000000,1.340000,1.080000,0.980000
A,0.910000,1.010000,2.000000,1.050000,1.110000
G,1.060000,0.980000,2.620000,0.960000,1.130000
GA,0.970000,0.970000,2.860000,0.990000,1.150000


[]

### EXP: Ablation Study for Parameter p


In [12]:
rs_imagenet3 = plot_eval_rs(
    *process_eval_rs(
        imagenet_result,
        imagenet_settings,
        k="sa",
        seg=True,
        force=False,
        include=[
            "GAG-1",
            "GAG-10",
            "GAG-9",
            "GAG-8",
            "GAG-7",
            "GAG-6",
            "GAG-5",
            "GAG-4",
            "GAG-3",
        ],
        use_include_order=True,
    ),
    sort_key=None,
    ascending=False,
    table_name="Ablation Study for Parameter p"
)

  0%|          | 0/9 [00:00<?, ?it/s]

[(1, 'GAG-1'), (2, 'GAG-10'), (3, 'GAG-9'), (4, 'GAG-8'), (5, 'GAG-7'), (6, 'GAG-6'), (7, 'GAG-5'), (8, 'GAG-4'), (9, 'GAG-3')]


Low MSE,MAE,Log-Cosh Dice Loss,RCAP2,DAUC,IAUC,Key,Params,Name
0.405191,0.509218,0.246309,0.308988,0.107451,0.684256,4,"x: abs, mean",GAG-1
0.402475,0.510331,0.241814,0.308646,0.107423,0.684055,4.i,"x: abs, 0.15 guided",GAG-10
0.403296,0.505856,0.240659,0.322326,0.107348,0.683809,4.h,"x: abs, 0.25 guided",GAG-9
0.403917,0.500605,0.238770,0.337917,0.107125,0.683852,4.g,"x: abs, 0.35 guided",GAG-8
0.405552,0.494825,0.237912,0.357661,0.106905,0.683572,4.f,"x: abs, 0.45 guided",GAG-7
0.408399,0.488934,0.238083,0.381494,0.106929,0.682545,4.e,"x: abs, 0.55 guided",GAG-6
0.412841,0.482750,0.240162,0.404181,0.106110,0.682881,4.d,"x: abs, 0.65 guided",GAG-5
0.421033,0.477539,0.247934,0.422528,0.105346,0.682472,4.c,"x: abs, 0.75 guided",GAG-4
0.435339,0.474767,0.267848,0.437721,0.104200,0.681461,4.b,"x: abs, 0.85 guided",GAG-3


In [19]:
gag_map_imagenet = {}
for r in rs_imagenet3["Prob: M3"][2]:
    gag_map_imagenet[r[0]] = r[1]
print(gag_map_imagenet)
# gag3 / gag1
round(gag_map_imagenet["GAG-3"] / gag_map_imagenet["GAG-1"], 4)

{'GAG-3': 0.43772091513686157, 'GAG-6': 0.3814937641877957, 'GAG-5': 0.40418100574461496, 'GAG-4': 0.4225278635818688, 'GAG-7': 0.3576606331054074, 'GAG-1': 0.30898786028064607, 'GAG-9': 0.32232592242373254, 'GAG-10': 0.30864555627316687, 'GAG-8': 0.33791738058653337}


1.4166

### ISIC


#### ISIC - All


In [16]:
isic_result, isic_settings = read_eval_rs(isic_settings, "sa", "isic", "effnet", "isic")

_ = plot_eval_rs(
    *process_eval_rs(
        isic_result,
        isic_settings,
        k="sa",
        seg=True,
        force=False,
        exclude=[
            "IG",
            "IG-MP",
            "Pos IG",
            "Neg IG",
            "Abs IG",
            "Proposed_Guided IG",
            "Abs Proposed_Guided IG",
        ],
    ),
    sort_key="RCAP2",
    ascending=False
)

  0%|          | 0/44 [00:00<?, ?it/s]

[(1, 'GradCAM'), (2, 'GAG-3'), (3, 'Proposed_Guided SmoothGrad'), (4, 'GAG-4'), (5, 'Abs Proposed_Guided BlurIG'), (6, 'Proposed_Guided BlurIG'), (7, 'Proposed_Guided GuidedIG'), (8, 'Proposed_Guided IG2'), (9, 'SmoothGrad-SQ'), (10, 'VarGrad'), (11, 'Abs Proposed_Guided GuidedIG'), (12, 'Abs Proposed_Guided IG2'), (13, 'GAG-5'), (14, 'GAG-6'), (15, 'GAG-7'), (16, 'Pos BlurIG'), (17, 'Neg BlurIG'), (18, 'Abs BlurIG'), (19, 'GAG-8'), (20, 'BlurIG'), (21, 'GAG-9'), (22, 'BlurIG-MP'), (23, 'Pos SmoothGrad'), (24, 'GAG-1'), (25, 'GAG-10'), (26, 'SmoothGrad'), (27, 'GuidedIG'), (28, 'Neg IG2'), (29, 'Neg SmoothGrad'), (30, 'IG-MP2'), (31, 'IG2'), (32, 'Pos IG2'), (33, 'GuidedIG-MP'), (34, 'Pos GuidedIG'), (35, 'SmoothGrad-MP'), (36, 'GB-MP'), (37, 'Abs GuidedIG'), (38, 'Neg GuidedIG'), (39, 'GB'), (40, 'Both GB'), (41, 'Abs GB'), (42, 'Vanilla'), (43, 'Neg GB'), (44, 'Abs IG2')]


RCAP,DAUC,IAUC,Name
0.641036,0.617663,0.789734,GradCAM
0.571426,0.462047,0.752674,GAG-3
0.570645,0.473559,0.750033,Proposed_Guided SmoothGrad
0.536391,0.463632,0.753085,GAG-4
0.525796,0.482848,0.765026,Abs Proposed_Guided BlurIG
0.523161,0.481121,0.764488,Proposed_Guided BlurIG
0.514245,0.484701,0.750181,Proposed_Guided GuidedIG
0.511599,0.502623,0.741231,Proposed_Guided IG2
0.510157,0.462224,0.752627,SmoothGrad-SQ
0.507876,0.463837,0.753282,VarGrad


#### EXP: Comparison with SOTA and Reversed


In [38]:
rs_isic1 = plot_eval_rs(
    *process_eval_rs(
        isic_result,
        isic_settings,
        k="sa",
        seg=True,
        force=False,
        include=[
            # "GradCAM",
            "GAG-3",
            "VarGrad",
            "SmoothGrad",
            "IG2",
            "GB",
            "GuidedIG",
            "BlurIG",
            "Vanilla",
            # "SmoothGrad-MP",
            # "IG-MP2",
            # "GB-MP",
            # "GuidedIG-MP",
            # "BlurIG-MP",
        ],
        # use_include_order=True,
    ),
    # sort_key=None,
    sort_key="RCAP↑",
    ascending=False,
    table_name="Comparison with SOTA and Reversed"
)

  0%|          | 0/8 [00:00<?, ?it/s]

[(1, 'Guided AbsoluatGrad'), (2, 'VarGrad'), (3, 'Blur IG'), (4, 'SmoothGrad'), (5, 'Guided IG'), (6, 'Integrated Gradients'), (7, 'Guided Backpropagation'), (8, 'Vanilla Gradient')]


RCAP↑,DAUC↓,IAUC↑,Name
0.571426,0.462047,0.752674,Guided AbsoluatGrad
0.507876,0.463837,0.753282,VarGrad
0.409689,0.479040,0.763821,Blur IG
0.364411,0.490366,0.744140,SmoothGrad
0.356746,0.493596,0.747145,Guided IG
0.344120,0.504229,0.740013,Integrated Gradients
0.323745,0.499964,0.741209,Guided Backpropagation
0.323745,0.499964,0.741209,Vanilla Gradient


#### EXP: Ablation Study for Different Gradient Process


In [23]:
rs_isic2 = plot_eval_rs(
    *process_eval_rs(
        isic_result,
        isic_settings,
        k="sa",
        seg=True,
        force=False,
        include=[
            "SmoothGrad",
            "Pos SmoothGrad",
            "Neg SmoothGrad",
            "GAG-1",
            "Proposed_Guided SmoothGrad",
            "GAG-3",
            "IG2",
            "Pos IG2",
            "Neg IG2",
            "Abs IG2",
            "Proposed_Guided IG2",
            "Abs Proposed_Guided IG2",
            "GuidedIG",
            "Pos GuidedIG",
            "Neg GuidedIG",
            "Abs GuidedIG",
            "Proposed_Guided GuidedIG",
            "Abs Proposed_Guided GuidedIG",
            "BlurIG",
            "Pos BlurIG",
            "Neg BlurIG",
            "Abs BlurIG",
            "Proposed_Guided BlurIG",
            "Abs Proposed_Guided BlurIG",
            "Both GB",
            "GB",
            "Neg GB",
            "Abs GB",
        ],
        use_include_order=True,
    ),
    sort_key=None,
    ascending=False,
    table_name="Ablation Study for Different Gradient Process"
)

  0%|          | 0/28 [00:00<?, ?it/s]

[(1, 'SmoothGrad'), (2, 'Pos SmoothGrad'), (3, 'Neg SmoothGrad'), (4, 'GAG-1'), (5, 'Proposed_Guided SmoothGrad'), (6, 'GAG-3'), (7, 'IG2'), (8, 'Pos IG2'), (9, 'Neg IG2'), (10, 'Abs IG2'), (11, 'Proposed_Guided IG2'), (12, 'Abs Proposed_Guided IG2'), (13, 'GuidedIG'), (14, 'Pos GuidedIG'), (15, 'Neg GuidedIG'), (16, 'Abs GuidedIG'), (17, 'Proposed_Guided GuidedIG'), (18, 'Abs Proposed_Guided GuidedIG'), (19, 'BlurIG'), (20, 'Pos BlurIG'), (21, 'Neg BlurIG'), (22, 'Abs BlurIG'), (23, 'Proposed_Guided BlurIG'), (24, 'Abs Proposed_Guided BlurIG'), (25, 'Both GB'), (26, 'GB'), (27, 'Neg GB'), (28, 'Abs GB')]


RCAP2,DAUC,IAUC,Key,Params,Name
0.364411,0.490366,0.744140,2_isic,"smooth: 015, no abs, no square",SmoothGrad
0.377659,0.466907,0.738956,13_isic,pos smooth: 015,Pos SmoothGrad
0.346149,0.488586,0.749321,14_isic,neg smooth: 015,Neg SmoothGrad
0.377488,0.450519,0.754289,4_isic,"x: abs, mean",GAG-1
0.570645,0.473559,0.750033,2.b_isic,"smooth: 015, no abs, x 85",Proposed_Guided SmoothGrad
0.571426,0.462047,0.752674,4.b_isic,"x: abs, 0.85 guided",GAG-3
0.344120,0.504229,0.740013,6_isic2,"intgrad: 5 step, 2 trails",IG2
0.341621,0.520616,0.751834,6.b_isic2,"intgrad: 5 step, 2 trails",Pos IG2
0.350858,0.474115,0.710033,6.a_isic2,"intgrad: 5 step, 2 trails",Neg IG2
0.320555,0.489816,0.747170,6.c_isic2,"intgrad: 10 step, 3 trails",Abs IG2


In [24]:
get_df(rs_isic2)

,RCAP,DAUC,IAUC
+,1.00,0.96,1.00
-,0.98,1.02,0.98
A,0.98,0.98,1.01
G,1.35,0.99,1.00
GA,1.35,0.98,1.00


#### EXP: Ablation Study for Parameter p


In [25]:
rs_isic3 = plot_eval_rs(
    *process_eval_rs(
        isic_result,
        isic_settings,
        k="sa",
        seg=True,
        force=False,
        include=[
            "GAG-1",
            "GAG-10",
            "GAG-9",
            "GAG-8",
            "GAG-7",
            "GAG-6",
            "GAG-5",
            "GAG-4",
            "GAG-3",
        ],
        use_include_order=True,
    ),
    sort_key=None,
    ascending=False,
    table_name="Ablation Study for Parameter p"
)

  0%|          | 0/9 [00:00<?, ?it/s]

[(1, 'GAG-1'), (2, 'GAG-10'), (3, 'GAG-9'), (4, 'GAG-8'), (5, 'GAG-7'), (6, 'GAG-6'), (7, 'GAG-5'), (8, 'GAG-4'), (9, 'GAG-3')]


RCAP2,DAUC,IAUC,Key,Params,Name
0.377488,0.450519,0.754289,4_isic,"x: abs, mean",GAG-1
0.374189,0.452034,0.754036,4.i_isic,"x: abs, 0.15 guided",GAG-10
0.391952,0.455257,0.754123,4.h_isic,"x: abs, 0.25 guided",GAG-9
0.411819,0.458050,0.753785,4.g_isic,"x: abs, 0.35 guided",GAG-8
0.437174,0.461553,0.753263,4.f_isic,"x: abs, 0.45 guided",GAG-7
0.466530,0.463375,0.753502,4.e_isic,"x: abs, 0.55 guided",GAG-6
0.500190,0.464263,0.753094,4.d_isic,"x: abs, 0.65 guided",GAG-5
0.536391,0.463632,0.753085,4.c_isic,"x: abs, 0.75 guided",GAG-4
0.571426,0.462047,0.752674,4.b_isic,"x: abs, 0.85 guided",GAG-3


In [26]:
gag_map_isic = {}
for r in rs_isic3["Prob: M3"][2]:
    gag_map_isic[r[0]] = r[1]
print(gag_map_isic)
# gag3 / gag1
round(gag_map_isic["GAG-3"] / gag_map_isic["GAG-1"], 4)

{'GAG-5': 0.5001895789101535, 'GAG-9': 0.3919519568132006, 'GAG-10': 0.3741892596409666, 'GAG-6': 0.46652970921015474, 'GAG-3': 0.5714262915718548, 'GAG-1': 0.3774878158676274, 'GAG-4': 0.5363908859623322, 'GAG-7': 0.4371741417984161, 'GAG-8': 0.4118185501051036}


1.5138

### Places365


#### Places365 - All


In [40]:
places365_result, places365_settings = read_eval_rs(
    places365_settings, "sa", "places365", "densenet161", "places365"
)

_ = plot_eval_rs(
    *process_eval_rs(
        places365_result,
        places365_settings,
        k="sa",
        seg=True,
        force=False,
        # k = 'sa', seg=True, force=True,
        exclude=["GAG-11", "GAG-12"],
    ),
    # sort_key="Score: M1", ascending = False,
    sort_key="RCAP↑",
    ascending=False
)

  0%|          | 0/44 [00:00<?, ?it/s]

[(1, 'GradCAM'), (2, 'Guided AbsoluatGrad'), (3, 'GAG-4'), (4, 'Proposed_Guided SmoothGrad'), (5, 'GAG-5'), (6, 'GAG-6'), (7, 'SmoothGrad-SQ'), (8, 'GAG-7'), (9, 'VarGrad'), (10, 'GAG-8'), (11, 'GAG-9'), (12, 'Guided Backpropagation'), (13, 'GAG-10'), (14, 'GAG-1'), (15, 'Abs Proposed_Guided BlurIG'), (16, 'Proposed_Guided BlurIG'), (17, 'Neg GB'), (18, 'Abs BlurIG'), (19, 'Pos SmoothGrad'), (20, 'Blur IG'), (21, 'Abs Proposed_Guided GuidedIG'), (22, 'Proposed_Guided GuidedIG'), (23, 'SmoothGrad'), (24, 'Abs GuidedIG'), (25, 'Abs Proposed_Guided IG'), (26, 'Pos BlurIG'), (27, 'Proposed_Guided IG'), (28, 'Abs IG'), (29, 'Guided IG'), (30, 'Neg BlurIG'), (31, 'Abs GB'), (32, 'Integrated Gradients'), (33, 'GB-MP'), (34, 'Pos GuidedIG'), (35, 'BlurIG-MP'), (36, 'Neg IG'), (37, 'Neg GuidedIG'), (38, 'Neg SmoothGrad'), (39, 'Both GB'), (40, 'Vanilla Gradient'), (41, 'SmoothGrad-MP'), (42, 'GuidedIG-MP'), (43, 'IG-MP'), (44, 'Pos IG')]


RCAP↑,DAUC↓,IAUC↑,Name
0.270829,0.293480,0.595581,GradCAM
0.156186,0.183537,0.501765,Guided AbsoluatGrad
0.151659,0.183435,0.500861,GAG-4
0.144982,0.172019,0.501721,Proposed_Guided SmoothGrad
0.144916,0.184098,0.501427,GAG-5
0.134830,0.184283,0.502009,GAG-6
0.131656,0.178531,0.511464,SmoothGrad-SQ
0.126252,0.183963,0.504265,GAG-7
0.122973,0.184234,0.494715,VarGrad
0.118414,0.183841,0.506977,GAG-8


#### EXP: Comparison with SOTA and Reversed


In [41]:
rs_places3651 = plot_eval_rs(
    *process_eval_rs(
        places365_result,
        places365_settings,
        k="sa",
        seg=True,
        force=False,
        include=[
            "GAG-3",
            "SmoothGrad",
            "VarGrad",
            "IG",
            "GB",
            "BlurIG",
            "GuidedIG",
            "Vanilla",
            # "IG-MP",
            # "GB-MP",
            # "GradCAM",
            # "SmoothGrad-MP",
            # "SmoothGrad-SQ",
            # "BlurIG-MP",
            # "GuidedIG-MP",
        ],
        # use_include_order=True,
    ),
    # sort_key=None,
    sort_key="RCAP↑",
    ascending=False,
    table_name="Comparison with SOTA and Reversed"
)

  0%|          | 0/8 [00:00<?, ?it/s]

[(1, 'Guided AbsoluatGrad'), (2, 'VarGrad'), (3, 'Guided Backpropagation'), (4, 'Blur IG'), (5, 'SmoothGrad'), (6, 'Guided IG'), (7, 'Integrated Gradients'), (8, 'Vanilla Gradient')]


RCAP↑,DAUC↓,IAUC↑,Name
0.156186,0.183537,0.501765,Guided AbsoluatGrad
0.122973,0.184234,0.494715,VarGrad
0.111551,0.160278,0.586063,Guided Backpropagation
0.060184,0.167668,0.488642,Blur IG
0.047968,0.147710,0.474288,SmoothGrad
0.030106,0.139110,0.443488,Guided IG
0.025042,0.155752,0.404904,Integrated Gradients
0.013201,0.256877,0.309799,Vanilla Gradient


#### EXP: Ablation Study for Different Gradient Process


In [29]:
rs_places3652 = plot_eval_rs(
    *process_eval_rs(
        places365_result,
        places365_settings,
        k="sa",
        seg=True,
        force=False,
        include=[
            "SmoothGrad",
            "Pos SmoothGrad",
            "Neg SmoothGrad",
            "GAG-1",
            "Proposed_Guided SmoothGrad",
            "GAG-3",
            "IG",
            "Pos IG",
            "Neg IG",
            "Abs IG",
            "Proposed_Guided IG",
            "Abs Proposed_Guided IG",
            "GuidedIG",
            "Pos GuidedIG",
            "Neg GuidedIG",
            "Abs GuidedIG",
            "Proposed_Guided GuidedIG",
            "Abs Proposed_Guided GuidedIG",
            "BlurIG",
            "Pos BlurIG",
            "Neg BlurIG",
            "Abs BlurIG",
            "Proposed_Guided BlurIG",
            "Abs Proposed_Guided BlurIG",
            "Both GB",
            "GB",
            "Neg GB",
            "Abs GB",
        ],
        use_include_order=True,
    ),
    sort_key=None,
    ascending=False,
    table_name="Comparison with SOTA and Reversed"
)

  0%|          | 0/28 [00:00<?, ?it/s]

[(1, 'SmoothGrad'), (2, 'Pos SmoothGrad'), (3, 'Neg SmoothGrad'), (4, 'GAG-1'), (5, 'Proposed_Guided SmoothGrad'), (6, 'GAG-3'), (7, 'IG'), (8, 'Pos IG'), (9, 'Neg IG'), (10, 'Abs IG'), (11, 'Proposed_Guided IG'), (12, 'Abs Proposed_Guided IG'), (13, 'GuidedIG'), (14, 'Pos GuidedIG'), (15, 'Neg GuidedIG'), (16, 'Abs GuidedIG'), (17, 'Proposed_Guided GuidedIG'), (18, 'Abs Proposed_Guided GuidedIG'), (19, 'BlurIG'), (20, 'Pos BlurIG'), (21, 'Neg BlurIG'), (22, 'Abs BlurIG'), (23, 'Proposed_Guided BlurIG'), (24, 'Abs Proposed_Guided BlurIG'), (25, 'Both GB'), (26, 'GB'), (27, 'Neg GB'), (28, 'Abs GB')]


RCAP2,DAUC,IAUC,Key,Params,Name
0.047968,0.147710,0.474288,2_places365,"smooth: 015, no th, no abs, no square",SmoothGrad
0.062934,0.182031,0.398714,13_places365,pos smooth: 015,Pos SmoothGrad
0.014383,0.158282,0.412865,14_places365,neg smooth: 015,Neg SmoothGrad
0.108280,0.181929,0.514909,4_places365,"x: abs, mean, Uniform",GAG-1
0.144982,0.172019,0.501721,2.b_places365,"smooth: 015, no abs, x 75",Proposed_Guided SmoothGrad
0.156186,0.183537,0.501765,4.b_places365,"x: abs, 0.85 guided",GAG-3
0.025042,0.155752,0.404904,6_places365,"intgrad: 5 step, 2 trails",IG
0.007908,0.144647,0.350411,6.b_places365,"intgrad: 5 step, 2 trails",Pos IG
0.016257,0.164060,0.347322,6.a_places365,"intgrad: 5 step, 2 trails",Neg IG
0.030566,0.153913,0.433238,6.c_places365,"intgrad: 5 step, 2 trails",Abs IG


In [30]:
get_df(rs_places3652)

,RCAP,DAUC,IAUC
+,2.26,0.89,1.10
-,1.73,1.11,0.98
A,1.64,1.03,1.12
G,3.20,0.97,1.20
GA,3.32,1.01,1.21


#### EXP: Ablation Study for Parameter p


In [31]:
rs_places36533 = plot_eval_rs(
    *process_eval_rs(
        places365_result,
        places365_settings,
        k="sa",
        seg=True,
        force=False,
        include=[
            "GAG-1",
            "GAG-10",
            "GAG-9",
            "GAG-8",
            "GAG-7",
            "GAG-6",
            "GAG-5",
            "GAG-4",
            "GAG-3",
        ],
        use_include_order=True,
    ),
    sort_key=None,
    ascending=False,
    table_name="Ablation Study for Parameter p"
)

  0%|          | 0/9 [00:00<?, ?it/s]

[(1, 'GAG-1'), (2, 'GAG-10'), (3, 'GAG-9'), (4, 'GAG-8'), (5, 'GAG-7'), (6, 'GAG-6'), (7, 'GAG-5'), (8, 'GAG-4'), (9, 'GAG-3')]


RCAP2,DAUC,IAUC,Key,Params,Name
0.108280,0.181929,0.514909,4_places365,"x: abs, mean, Uniform",GAG-1
0.108537,0.182319,0.512090,4.i_places365,"x: abs, 0.15 guided",GAG-10
0.113312,0.183024,0.509891,4.h_places365,"x: abs, 0.25 guided",GAG-9
0.118414,0.183841,0.506977,4.g_places365,"x: abs, 0.35 guided",GAG-8
0.126252,0.183963,0.504265,4.f_places365,"x: abs, 0.45 guided",GAG-7
0.134830,0.184283,0.502009,4.e_places365,"x: abs, 0.55 guided",GAG-6
0.144916,0.184098,0.501427,4.d_places365,"x: abs, 0.65 guided",GAG-5
0.151659,0.183435,0.500861,4.c_places365,"x: abs, 0.75 guided",GAG-4
0.156186,0.183537,0.501765,4.b_places365,"x: abs, 0.85 guided",GAG-3


In [32]:
gag_map_places365 = {}
for r in rs_places36533["Prob: M3"][2]:
    gag_map_places365[r[0]] = r[1]
print(gag_map_places365)
# gag3 / gag1
round(gag_map_places365["GAG-3"] / gag_map_places365["GAG-1"], 4)

{'GAG-3': 0.1561861581529511, 'GAG-5': 0.1449156624405223, 'GAG-10': 0.10853673090046367, 'GAG-4': 0.15165889547561842, 'GAG-6': 0.13482986743435388, 'GAG-9': 0.11331194987864397, 'GAG-1': 0.10828008063024988, 'GAG-7': 0.1262523289988132, 'GAG-8': 0.1184144722800215}


1.4424